In [2]:
import automation1 as a1
import time
import numpy as np
import matplotlib.pyplot as plt

import os
import serial
import time

import math

ModuleNotFoundError: No module named 'automation1'

In [1]:
## make sure is available on Saw1 computer
from pathlib import Path

# Connect to Controller

In [2]:
controller = a1.Controller.connect()
controller.start()
print(controller.is_running)

KeyboardInterrupt: 

# A. Setup & Constants

In [3]:
base_path = Path(r"C:\Users\UNIVERSITY\Desktop\RunData\CCAT_Alumina_350\Surface2\90deg\TestCuts\SpindleC\CutCammingThin")
# Filenames
cuttype     = "Thin"                # Equivalent to #define cuttype "Thin"
mastername  = "Master.txt"          # The list of cuts
datafile    = "DataCollectCut.dat"  # Data collection output
outfile     = "MetCheck.txt"        # Metrology check output
lockname    = "lockfile.lock"       # Lockfile to prevent re-running

In [4]:
# Parameters
safelift        = 40.0   # 20 mm above z-start for safe moves
numpoints       = 10     # number of points (if doing metrology checks)
lifter_settle_s = 0.5    # seconds
feedspeed       = 5.0   # 11 mm/s 

# Build full file paths (Path objects)
master_path = base_path / mastername
data_path   = base_path / datafile
out_path    = base_path / outfile
lock_path   = base_path / lockname

# Just to verify in notebook
print("Base path: ", base_path)
print("Master file: ", master_path)
print("Lock file: ", lock_path)

Base path:  C:\Users\UNIVERSITY\Desktop\RunData\CCAT_Alumina_350\Surface2\90deg\TestCuts\SpindleC\CutCammingThin
Master file:  C:\Users\UNIVERSITY\Desktop\RunData\CCAT_Alumina_350\Surface2\90deg\TestCuts\SpindleC\CutCammingThin/Master.txt
Lock file:  C:\Users\UNIVERSITY\Desktop\RunData\CCAT_Alumina_350\Surface2\90deg\TestCuts\SpindleC\CutCammingThin/lockfile.lock


# B. Initial Motions, Digital Output (DO), lockfile check

In [ ]:
lock_path = base_path / lockname
if lock_path.exists():
    print(f"[guard] Lockfile present at {lock_path} — stopping here.")
    ### Need to actually add a stopping component if putting this in script form ###

In [ ]:
# start a queue with capacity 64, and block if full
cq = controller.runtime.commands.begin_command_queue(task=1, command_capacity=64, should_block_if_full=True)
print("Queue started on task:", cq.task_index, "capacity:", cq.command_capacity)


In [ ]:
cq.pause()

In [ ]:
# queued versions of the same calls you used before
cq.commands.moveabsolute(["ZA"], [0.0],     [20.0])
cq.commands.moveabsolute(["ZB"], [0.0],     [20.0])
cq.commands.moveabsolute(["ZC"], [-0.0005], [20.0])
cq.commands.waitforinposition(["ZA","ZB","ZC"])


In [ ]:
## turn on the digital outputs and set the dwell in the queue
io_axis_misc  = "X"  # adjust if your IO lives elsewhere
io_axis_flood = "X"

cq.commands.digitaloutputset(axis=io_axis_misc,  output_num=10, value=1)
cq.commands.digitaloutputset(axis=io_axis_flood, output_num=6,  value=1)
cq.commands.movedelay(["X","Y","ZC"], delay_time=11_000)  # 11 s


In [ ]:
cq.resume()   
cq.wait_for_empty()  # block until all queued commands above finish


controller.runtime.commands.end_command_queue(cq)

#### ZC Drive Status Check 

In [ ]:
cfg = a1.StatusItemConfiguration()
cfg.axis.add(a1.AxisStatusItem.DriveStatus,     "ZC")
cfg.axis.add(a1.AxisStatusItem.ProgramPosition, "ZC")

results = controller.runtime.status.get_status_items(cfg)

# pull the drive status (bitfield) and program position (float)
drive_status = int(results.axis.get(a1.AxisStatusItem.DriveStatus, "ZC"))
program_pos  = results.axis.get(a1.AxisStatusItem.ProgramPosition, "ZC")

# mask off camming bit (bit 16)
camming_bit = bool(drive_status & (1 << 16))

print(f"[diag] ZC DriveStatus = 0x{drive_status:08X}")
print(f"[diag] ZC camming bit set? {camming_bit}")
print(f"[diag] ZC ProgramPosition = {program_pos:.4f} mm")

# C. Open Master.txt File and turn on Flood Cooling 

In [5]:
assert master_path.exists(), f"Master file not found: {master_path}"
print("[step C] Found Master.txt:", master_path)

with open(master_path, "r") as f:
    raw_lines = [ln.strip() for ln in f if ln.strip()]

print(f"[step C] Read {len(raw_lines)} line(s) from Master.txt.")
if raw_lines:
    print(" first line →", raw_lines[0])

AssertionError: Master file not found: C:\Users\UNIVERSITY\Desktop\RunData\CCAT_Alumina_350\Surface2\90deg\TestCuts\SpindleC\CutCammingThin/Master.txt

In [ ]:
# Flood cooling ON (DO6 = 1)
io_axis_flood = "X"  # change if flood IO lives elsewhere
controller.commands.io.digitaloutputset(axis=io_axis_flood, output_num=6, value=1, execution_task_index=1)
print("Flood coolant ON")

# Dwell 11 s (same as AeroBasic DWELL 11.0)
controller.commands.motion.movedelay(["X","Y","ZC"], delay_time=11_000, execution_task_index=1)


In [ ]:
# parse the first cut definition
# AeroBasic FILEREAD expected 5 numeric values per row
# camnum  xvalue  ystart  zstart  yend
row0 = lines[0].split()
assert len(row0) >= 5, f"Expected ≥5 fields, got {len(row0)}: {row0}"

camnum  = int(row0[0])
xvalue  = float(row0[1])
ystart  = float(row0[2])
zstart  = float(row0[3])
yend    = float(row0[4])

print(f"[step C] camnum={camnum}, x={xvalue}, ystart={ystart}, zstart={zstart}, yend={yend}")

# these two mirror variables the AeroBasic computed (useful later if we do met checks)
#yval = ystart
#dy   = (yend - ystart) / numpoints if numpoints else 0.0


#print(f"[step C] yval={yval}, dy={dy}")


In [ ]:
## build the cam table file name for this row and check it exists
cam_filename = f"CutCam{cuttype}{camnum:04d}.Cam"
cam_path     = base_path / cam_filename

print("[step C] Cam file:", cam_path)
if not cam_path.exists():
    print("Cam file not found.")



# D. Main Loop over Master.txt Rows

In [ ]:
assert cam_path.exists(), f"Cam file not found: {cam_path}"
print("[step D] Using cam file:", cam_path)

with open(cam_path, "r") as f:
    shown = 0
    for ln in f:
        s = ln.strip()
        if not s or s.startswith(("#",";")):
            continue
        print(" cam>", s)
        shown += 1
        if shown >= 5:
            break

In [ ]:
leader_values = []
follower_values = []

with open(cam_path, "r") as f:
    for ln in f:
        s = ln.strip()
        if not s or s.startswith(("#",";")):
            continue
        parts = s.replace(",", " ").split()
        if len(parts) < 2:
            continue
        leader_values.append(float(parts[0]))    # Y
        follower_values.append(float(parts[1]))  # ZC

n = len(leader_values)
assert n > 1, "Parsed too few cam points."
print(f"[step D] Parsed {n} cam points.")

In [ ]:
from a1.public.enums_gen import (
    CammingUnits, CammingInterpolation, CammingWrapMode
)


In [ ]:
####### initiate advanced motion module ############
am = controller.commands.advanced_motion

In [ ]:
# free table 1 in case something is already loaded
am.cammingfreetable(1, execution_task_index=1)

In [ ]:
am.cammingloadtablefromarray(
    table_num=1,
    leader_values=leader_values,
    follower_values=follower_values,
    num_values=len(leader_values),
    units_mode=CammingUnits.Position,               # follower is position vs leader
    interpolation_mode=CammingInterpolation.Linear, # typical for .Cam
    wrap_mode=CammingWrapMode.NoWrap,               # NOWRAP
    table_offset=0.0,
    execution_task_index=1
)
print("[step D] Camming table 1 loaded.")

In [6]:
# slow, safe test speeds
SPEED_Y_TRAVERSE  = 5.0   # mm/s
SPEED_X_TRAVERSE  = 5.0   # mm/s
SPEED_ZC_APPROACH = 2.0   # mm/s  (down to zstart+2)
SPEED_ZC_TOUCH    = 0.5   # mm/s  (final settle at zstart)


In [ ]:
# begin a new queue 
cq = controller.runtime.commands.begin_command_queue(task=1, command_capacity=64, should_block_if_full=True)
print(" queue started.")

In [ ]:
# move y to start position, wait for in position
cq.commands.moveabsolute(["Y"], [ystart], [SPEED_Y_TRAVERSE])
cq.commands.waitforinposition(["Y"])
print("queued Y→ystart + wait.")

In [ ]:
## move X 
cq.commands.moveabsolute(["X"], [xvalue], [SPEED_X_TRAVERSE])
print("[D-Q4] queued X→xvalue.")

In [ ]:
## move ZC --- do we need to do this? yes
cq.commands.moveabsolute(["ZC"], [zstart + 2.0], [SPEED_ZC_APPROACH])
cq.commands.moveabsolute(["ZC"], [zstart],        [SPEED_ZC_TOUCH])
cq.commands.waitforinposition(["ZC"])

In [ ]:
from automation1.public.enums_gen import CammingSource, CammingOutput

cq.commands.advanced_motion.cammingon(
    follower_axis="ZC",
    leader_axis="Y",
    table_num=1,
    source=CammingSource.Position,                    # leader uses position
    output=CammingOutput.SynchronousFollowerRelative  # matches CAMSYNC ...,1
)
print("[D-Q6] queued camming ON (ZC follows Y, relative).")

In [ ]:
cq.wait_for_empty()
print("[D-Q7] queue empty — verifying camming bit...")

In [ ]:
drive_status = int(results.axis.get(a1.AxisStatusItem.DriveStatus, "ZC"))
camming_bit  = bool(drive_status & (1 << 16))  # INDEXTOMASK(16)

print(f"[diag] ZC DriveStatus = 0x{drive_status:08X}")
print(f"[diag] ZC camming bit set? {camming_bit}")

In [7]:
#### once camming bit set is True, we're right where the Aerobasic command would say CAMSYNC. so next steps is to check the spindle 

In [ ]:
## checking if spindle speed is 0
io_axis_spindle   = "X"   # axis/module hosting the DI bank
spindle_input_num = 0     # from AeroBasic: DI[0]

val = controller.commands.io.digitalinputget(
    axis=io_axis_spindle, input_num=spindle_input_num, execution_task_index=1
)
print(f"[spindle] {io_axis_spindle}.DI[{spindle_input_num}] = {val}  (1 ⇒ Spindle Speed 0)")

if val == 1:
    raise RuntimeError("Spindle Speed 0 — aborting before lead-in.")


In [ ]:
### slow test speeds for cut camming right now

LEAD_IN_DIST     = 15.0   # mm
LEAD_IN_SPEED    = 1.0    # mm/s
FEED_SPEED_TEST  = 2.0    # mm/s
SPEED_ZC_RETRACT = 5.0    # mm/s


In [ ]:
# lead-in: Y from ystart → ystart + 15 mm (slow)
cq.commands.moveabsolute(["Y"], [ystart + LEAD_IN_DIST], [LEAD_IN_SPEED])


In [ ]:
# cut feed: Y to yend (slow test feed)
cq.commands.moveabsolute(["Y"], [yend], [FEED_SPEED_TEST])
cq.commands.waitforinposition(["Y"])


In [ ]:
# cam OFF (AeroBasic: SYNC ZC 1 0)
cq.commands.advanced_motion.cammingoff(follower_axis="ZC")


In [ ]:
# retract ZC and free table 1
cq.commands.moveabsolute(["ZC"], [zstart + safelift], [SPEED_ZC_RETRACT])
cq.commands.waitforinposition(["ZC"])
cq.commands.advanced_motion.cammingfreetable(1)


In [ ]:
# drain the queued work
cq.wait_for_empty()
print("[post-cut] lead-in, cut, cam OFF, ZC retract, free table — done.")

In [ ]:
## verify camming is off
drive_status = int(results.axis.get(a1.AxisStatusItem.DriveStatus, "ZC"))
camming_bit  = bool(drive_status & (1 << 16))
print(f"[diag] ZC camming bit after cammingoff? {camming_bit}")

In [ ]:
# Keep or create a global/current row index i
######## haven't created a loop or a function yet because just testing raw steps ########
######## at the moment to make sure everything makes sense first #######
try:
    i
except NameError:
    i = 0  # if not defined yet, start at first row already used

i += 1  # advance to next row
if i >= len(raw_lines):
    print("[next] No more cuts — reached end of Master.txt")
else:
    row = raw_lines[i].split()
    assert len(row) >= 5, f"Expected 5 fields on line {i}, got {len(row)}: {row}"

    camnum  = int(row[0])
    xvalue  = float(row[1])
    ystart  = float(row[2])
    zstart  = float(row[3])
    yend    = float(row[4])

    cam_filename = f"CutCam{cuttype}{camnum:04d}.Cam"
    cam_path     = base_path / cam_filename

    print(f"[next] i={i} camnum={camnum}  x={xvalue}  ystart={ystart}  zstart={zstart}  yend={yend}")
    print(f"[next] cam file → {cam_path}")
    # Now repeat Step D: parse cam file → load table → stage → cammingon → spindle check → lead-in & cut → cammingoff → retract → free.

# Cuts are Done from Master.txt File

In [ ]:
# drain and end any open queues on task 1
for qname in ("cq"): #, "cq2"):
    try:
        q = globals().get(qname)
        if q is not None:
            try:
                q.wait_for_empty()
            except Exception:
                pass
            controller.runtime.commands.end_command_queue(q)
            print(f"[E] ended queue: {qname}")
    except Exception as e:
        print(f"[E] (note) could not end {qname}: {e}")

In [ ]:
# flood coolant off (D06=0)
io_axis_flood = "X"  # change if your flood IO lives elsewhere
controller.commands.io.digitaloutputset(axis=io_axis_flood, output_num=6, value=0, execution_task_index=1)
print("flood coolant OFF (DO6=0).")


In [ ]:
### no command_queue okay??
motion = controller.commands.motion
motion.moveabsolute(["ZC"], [-0.0005], [20.0], execution_task_index=1)
motion.waitforinposition(["ZC"], execution_task_index=1)
print("[E] ZC parked at -0.0005.")

In [ ]:
### create lockfile
import datetime as _dt
lock_path = base_path / lockname
msg = f"The cutting has been completed, directory is now locked ({_dt.datetime.now().isoformat(timespec='seconds')})"

with open(lock_path, "w", encoding="utf-8") as f:
    f.write(msg + "\n")

print(f"[E] wrote lockfile → {lock_path}")

In [ ]:
###### still going to leave spindle running by the way

# End the Queue

In [ ]:
controller.runtime.commands.end_command_queue(cq)

# Command Queue Python Module

WARNINGS FROM THE DOCUMENTATION:
- must keep command queue populated at all times. If the command queue is not populated, your process will stall and motion problems might occur
- if a `MovePt()` or `MovePvt()` command is the most recently executed command from the queue, and a starvation of the command queue has occurred, the controlloer **will not** automatically decelerate the axes that you specified to a comand to zero velocity 

NOTES FROM THE DOCUMENTATION
- when the command queue begins on a task, the controller executes all the commands that are in the command queue as quickly as possible. If the controller automatically executes commands from the command queue more quickly than you can add them, the command queue will not have a sufficient quantity of commands to execute. This condition is known as a starvation of the command queue. 
- while the command queue is active, you can examine its status to find the number of times that a starvation of the command queue has occurred. 
- if starvation mode of the queue occurs, velocity blending mode is enabled on the task, AND most recently executed command from the command queue is a `MoveCcw()`, `MoveCw()`, or `MoveLinear()` command then the controller automatically decelerates them to zero velocity to prevent motion problems from occurring 
- can use `CommandQueueCount` task status item to make the first command wait for the command queue to fill with a specified number of commands before the controller executes them.

RELEVANT LINK: 
- http://help.aerotech.com/automation1/Content/APIs/Python/References/Command-Queue-Python.htm?Highlight=advanced_motion

# Status Command Queue Commands to Have in Arsenal

In [ ]:
command_queue.status.number_of_times.emptied

In [ ]:
command_queue.execute("wait(StatusGetTaskItem ... >= 20)")

# Important thoughts to factor in
1. Make sure UI threading is not going to cause problems with motion. Be careful in understanding this deeply. https://help.aerotech.com/automation1/Content/APIs/Python/Get-Started/Guidelines-Python.htm
2. add a stopping command execution component if lockfile present. right now, it's just a print statement.
3. Two ways to use the queue (both OK):
“Stream & drain”: enqueue → cq.wait_for_empty() → enqueue more → … → end_command_queue(cq).
“Arm & go”: cq.pause() → enqueue everything → cq.resume() → cq.wait_for_empty() → end_command_queue(cq).
4. have we lost the feedspeed anywhere in this code?
5. 